In [1]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt

def plot_yield_scaler(run, current, correction):
    filename = f"/lustre24/expphy/cache/hallc/c-nps/analysis/pass1/replays/skim/nps_hms_skim_{run}_1_-1.root"
    
    if not os.path.isfile(filename):
        print(f"Error: File not found for run {run}")
        return -1

    data_file = ROOT.TFile.Open(filename)
    scaler_tree = data_file.Get("TSH")
    if not scaler_tree:
        print(f"Error: Couldn't find scaler tree 'TSH' in file {filename}")
        return -1

    # Set up variables to read
    H_BCM4A_scalerCharge = ROOT.Double(0)
    H_BCM4A_scalerCurrent = ROOT.Double(0)
    H_EDTM_scaler = ROOT.Double(0)
    H_hTRIG4_scaler = ROOT.Double(0)
    H_1MHz_scalerTime = ROOT.Double(0)

    scaler_tree.SetBranchAddress("H.BCM4A.scalerCharge", ROOT.AddressOf(H_BCM4A_scalerCharge))
    scaler_tree.SetBranchAddress("H.BCM4A.scalerCurrent", ROOT.AddressOf(H_BCM4A_scalerCurrent))
    scaler_tree.SetBranchAddress("H.EDTM.scaler", ROOT.AddressOf(H_EDTM_scaler))
    scaler_tree.SetBranchAddress("H.hTRIG4.scaler", ROOT.AddressOf(H_hTRIG4_scaler))
    scaler_tree.SetBranchAddress("H.1MHz.scalerTime", ROOT.AddressOf(H_1MHz_scalerTime))

    nentries = scaler_tree.GetEntries()
    if nentries < 2:
        print(f"Error: Not enough scaler entries in run {run}")
        return -1

    accumulated_charge = 0
    accumulated_edtm = 0
    accumulated_hTRIG4 = 0
    prev_time = 0
    prev_EDTM = 0
    prev_hTRIG4 = 0

    for i in range(nentries):
        scaler_tree.GetEntry(i)

        if i == 0:
            prev_time = H_1MHz_scalerTime
            prev_EDTM = H_EDTM_scaler
            prev_hTRIG4 = H_hTRIG4_scaler
            continue

        delta_time = H_1MHz_scalerTime - prev_time

        if abs(H_BCM4A_scalerCurrent - current) < 1.5:
            corr_factor = (H_BCM4A_scalerCurrent + correction) / H_BCM4A_scalerCurrent
            corrected_current = H_BCM4A_scalerCurrent * corr_factor
            accumulated_charge += corrected_current * delta_time
            accumulated_edtm += (H_EDTM_scaler - prev_EDTM)
            accumulated_hTRIG4 += (H_hTRIG4_scaler - prev_hTRIG4)

        prev_time = H_1MHz_scalerTime
        prev_EDTM = H_EDTM_scaler
        prev_hTRIG4 = H_hTRIG4_scaler

    data_file.Close()

    if accumulated_charge <= 0:
        print(f"Warning: Accumulated charge is zero or negative for run {run}")
        return -1

    yield_norm = (accumulated_hTRIG4 - accumulated_edtm) / accumulated_charge
    return yield_norm

# ------------ Analysis Driver ------------
runs = [7003, 7004, 7005, 7006, 7007]  # carbon 3
currents = [40, 30, 20, 10, 5]         # carbon 3
correction = -0.1

yields = []
valid_currents = []

for run, current in zip(runs, currents):
    y = plot_yield_scaler(run, current, correction)
    if y > 0:
        print(f"Run {run}: Yield/Charge = {y}")
        yields.append(y)
        valid_currents.append(current)

# Normalize
min_yield = min(yields)
normalized_yields = [y / min_yield for y in yields]

# ------------ Plotting ------------
plt.figure(figsize=(8, 6))
plt.scatter(valid_currents, normalized_yields, color='red', label="Normalized Yield", marker='o')
plt.title("Yield vs Current (Normalized)")
plt.xlabel("Current [μA]")
plt.ylabel("Yield / Charge (normalized)")
plt.grid(True)
plt.legend()
plt.gca().invert_xaxis()  # Optional: invert x-axis to match some hall convention
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'ROOT'